**BBM473 Database Laboratory (Spring 2022)**


Exercise 6: Indexes
===========

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [ ]:
# IF YOU ARE RUNNING ON COLAB
# DOWNLOAD THE DATABASE
# ELSE MAKE SURE YOUR DATABASE 
# FILE IS IN THE SAME DIRECTORY 
# WITH YOUR NOTEBOOK FILE

!wget https://github.com/alibtasdemir/Exercise6/raw/main/complaint.db

--2022-04-06 13:05:40--  https://github.com/alibtasdemir/Exercise6/raw/main/complaint.db
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alibtasdemir/Exercise6/main/complaint.db [following]
--2022-04-06 13:05:40--  https://raw.githubusercontent.com/alibtasdemir/Exercise6/main/complaint.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22255616 (21M) [application/octet-stream]
Saving to: ‘complaint.db’

complaint.db        100%[===================>]  21.22M  --.-KB/s    in 0.07s   

2022-04-06 13:05:42 (291 MB/s) - ‘complaint.db’ saved [22255616/22255616]



Activate sql function and connect to the database

In [ ]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists state_product_index;
%sql drop index if exists state_index;
%sql analyze

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///complaint.db
Done.
 * sqlite:///complaint.db
Done.
 * sqlite:///complaint.db
Done.


[]

Number of records in the database. (It should be 79468)

In [ ]:
%sql select count(*) from complaints;

 * sqlite:///complaint.db
Done.


count(*)
79468


Take a look into the database

In [ ]:
%sql select * from complaints limit 5;

 * sqlite:///complaint.db
Done.


Date_received,Product,Subproduct,Issue,Subissue,Consumer_narrative,Company_public_response,Company,State,ZIP_code,Submitted_via,Date_sent_to_company,Company_response,Timely_response,Consumer_disputed,Complaint_ID
12/30/2015,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",,,Company chooses not to provide a public response,U.S. Bancorp,TN,38138,,N/A,Referral,01/05/2016,Closed with explanation,Yes
12/12/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",,,Company chooses not to provide a public response,Citibank,NY,13021,,N/A,Referral,12/23/2015,Closed with explanation,Yes
12/02/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",,,,Nationstar Mortgage,MI,49102,,N/A,Referral,12/17/2015,Closed with explanation,Yes
12/02/2015,Bank account or service,Other bank product/service,"Account opening, closing, or management",,,Company chooses not to provide a public response,Wells Fargo & Company,,,,N/A,Referral,12/07/2015,Closed with explanation,Yes
12/22/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",,,,Nationstar Mortgage,FL,33484,Older American,Consent not provided,Web,01/06/2016,Closed with explanation,Yes


### Task 1: Query without an index (40 pts.)

First, let's start off by writing a query to find the **counts of the top 5 Product, State pairs** in the complaints database (return the product and state as well as the count).  Use the single-line syntax for simple timing so we can see how long the query takes:

In [ ]:
%time %sql SELECT product, state, count(*) AS count FROM complaints GROUP BY product, state ORDER BY count DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 18.8 ms, sys: 3.16 ms, total: 22 ms
Wall time: 22.4 ms


Product,State,count
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 2: Single search key index (30 pts.)

Now create a _single-key_ index such that the above query is faster!  The syntax to create an index in SQL is:
> DROP INDEX IF EXISTS index_name;
> CREATE INDEX index_name ON table(attributes);

In [ ]:
%%sql
DROP INDEX IF EXISTS state_index;
CREATE INDEX state_index ON complaints(state);

 * sqlite:///complaint.db
Done.
Done.


[]

In [ ]:
%time %sql SELECT product, state, count(*) AS count FROM complaints GROUP BY product, state ORDER BY count DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 20.4 ms, sys: 1.85 ms, total: 22.3 ms
Wall time: 23 ms


Product,State,count
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 3 (a) (15 pts.):

Now, create a _covering_ index for the query and then see how long it takes to run!

In [ ]:
%%sql 
DROP INDEX IF EXISTS state_product_index;
CREATE INDEX state_product_index ON complaints(product, state);

 * sqlite:///complaint.db
Done.
Done.


[]

In [ ]:
%time %sql SELECT product, state, count(*) AS count FROM complaints GROUP BY product, state ORDER BY count DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 18.8 ms, sys: 1.87 ms, total: 20.7 ms
Wall time: 24.1 ms


Product,State,count
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 3 (b) (15 pts.):

Use EXPLAIN to see if sqlite used/recognized your covering index.  EXPLAIN is an operator that tells SQL to explain its query plan... we'll look into this in more depth later.  For now, the syntax is:
> EXPLAIN QUERY PLAN your_query_here;

In [ ]:
%%sql
EXPLAIN QUERY PLAN
SELECT product, state, count(*) AS c
FROM complaints
GROUP BY product, state ORDER BY c DESC LIMIT 5;

 * sqlite:///complaint.db
Done.


selectid,order,from,detail
0,0,0,SCAN TABLE complaints USING COVERING INDEX state_product_index
0,0,0,USE TEMP B-TREE FOR ORDER BY
